# Exemple de notebook - filtres sql et récupération des données en pandas

L'objectif de ce notebook est de fournir des exemples pour pré-filtrer les données via sql avant de charger les données dans un DataFrame pandas. 


In [2]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données.

import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

## Filtres
1. Filtrer sur les prélévements de 2024

In [13]:
query_2024 = """
SELECT * from edc_prelevements
WHERE dateprel >= '2024-01-01'
"""

prelevements_2024 = con.sql(query_2024)
prelevements_2024_df = prelevements_2024.df()
prelevements_2024_df.head(2)

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024


2. Filtrer sur les prélévements non conformes en 2024


In [16]:
where_clause = """
"dateprel" >= '2024-01-01'
  AND (
    (
      "plvconformitebacterio" = 'N'
    )
    OR (
      "plvconformitechimique" = 'N'
    )
    OR (
      "plvconformitereferencebact" = 'N'
    )
    OR (
      "plvconformitereferencechim" = 'N'
    )
  )
"""
query_non_conforme = f"""
SELECT
  *
FROM "edc_prelevements"
WHERE
  {where_clause}
"""
prelevements_2024_non_conforme = con.sql(query_non_conforme)
prelevements_2024_non_conforme_df = prelevements_2024_non_conforme.df()
prelevements_2024_non_conforme_df.head(2)

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition
0,001,001000003,01007,AMBRONAY,None,None,None,00100142530,2024-10-18,13h18,Eau d'alimentation non-conforme aux exigences ...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,N,C,N,C,2024
1,001,001000244,01268,MURS-ET-GELIGNIEUX,001000242,TTP (UV) MURS-ET-GELIGNIEUX,100 %,00100141265,2024-07-10,11h59,Eau d'alimentation non-conforme aux exigences ...,CC BUGEY SUD MURS-ET-GELIGNEUX,RÉGIE DES EAUX -CC BUGEY SUD,RÉGIE DES EAUX -CC BUGEY SUD,N,C,N,C,2024


## Selectionner des colonnes avant d'exécuter la requête

Selectionner les colonnes avant de charger les données permets une exécution plus rapide et limite l'usage de la mémoire.

In [18]:
query_preselected = f"""
SELECT
  "referenceprel",
  "dateprel",
  "nomcommuneprinc",
  "plvconformitebacterio"
FROM "edc_prelevements"
WHERE
  {where_clause}
"""
preselected = con.sql(query_preselected)
preselected_df = preselected.df()
preselected_df.head(2)

,referenceprel,dateprel,nomcommuneprinc,plvconformitebacterio
0,00100142530,2024-10-18,AMBRONAY,N
1,00100141265,2024-07-10,MURS-ET-GELIGNIEUX,N


## Jointure
Joindre edc_prelevements et edc_resultats sur referenceprel pour obtenir les résultats associés à chaque prélèvement :

In [39]:

query =  f"""
SELECT
  "edc_prelevements"."referenceprel",
  "edc_prelevements"."dateprel",
  "edc_prelevements"."nomcommuneprinc",
  "edc_resultats"."libmajparametre",
  "edc_resultats"."insituana",
  "edc_resultats"."rqana",
  "edc_resultats"."cdunitereferencesiseeaux"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) AS edc_prelevements
INNER JOIN "edc_resultats"
  ON "edc_prelevements"."referenceprel" = "edc_resultats"."referenceprel"
"""


joined = con.sql(query)
joined_df = joined.df()
joined_df

,referenceprel,dateprel,nomcommuneprinc,libmajparametre,insituana,rqana,cdunitereferencesiseeaux
0,00800105683,2024-08-19,VILLERS-SUR-BAR,ASPECT (QUALITATIF),L,Aspect normal,SANS OBJET
1,00800105683,2024-08-19,VILLERS-SUR-BAR,BACT. ET SPORES SULFITO-RÉDU./100ML,L,0,n/(100mL)
2,00800105683,2024-08-19,VILLERS-SUR-BAR,CONDUCTIVITÉ À 25°C,L,580,µS/cm
3,00800105683,2024-08-19,VILLERS-SUR-BAR,CHLORE LIBRE,T,"<0,05",mg(Cl2)/L
4,00800105683,2024-08-19,VILLERS-SUR-BAR,CHLORE TOTAL,T,"0,07",mg(Cl2)/L
...,...,...,...,...,...,...,...
6024762,97200106851,2024-01-15,GROS-MORNE,PH,L,"6,8",unité pH
6024763,97200106851,2024-01-15,GROS-MORNE,PH,T,"6,9",unité pH
6024764,97200106851,2024-01-15,GROS-MORNE,SAVEUR (QUALITATIF),T,Aucun changement anormal,SANS OBJET
6024765,97200106851,2024-01-15,GROS-MORNE,TEMPÉRATURE DE L'EAU (DOM),T,"24,3",°C


## Groupby et aggregats
Nombre total de prélèvements non conforme par commune en 2024

In [46]:
query = f"""
SELECT
  "nomcommuneprinc",
  COUNT("referenceprel") AS "nb_prelevements_non_conformes"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) 
GROUP BY
  1
"""
grouped = con.sql(query)
grouped_df = grouped.df()
grouped_df.sort_values("nb_prelevements_non_conformes", ascending=False)

,nomcommuneprinc,nb_prelevements_non_conformes
4857,MARSEILLE,2016
7041,CAPESTERRE-BELLE-EAU,592
6890,AJACCIO,374
7051,SAINT-DENIS,344
10496,PREMESQUES,294
...,...,...
10142,FLAGEY-LES-AUXONNE,1
10144,VILLERS-LA-FAYE,1
10146,SAINT-PEVER,1
4287,LADOYE-SUR-SEILLE,1


## Autres exemples :

In [9]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb
# Faisons une requête SQL en utilisant duckdb via la librarie python pour lister les substances qui ont été recherchées
# et les trier par ordre décroissant de leur nombre d'occurrences

con.sql("""
    SELECT libmajparametre, COUNT(*) as count
    FROM sise_resultats
    GROUP BY libmajparametre
    ORDER BY count DESC
""").show()

┌─────────────────────────────────────┬────────┐
│           libmajparametre           │ count  │
│               varchar               │ int64  │
├─────────────────────────────────────┼────────┤
│ PH                                  │ 338786 │
│ CONDUCTIVITÉ À 25°C                 │ 265188 │
│ TEMPÉRATURE DE L'EAU                │ 258858 │
│ CHLORE LIBRE                        │ 257631 │
│ ESCHERICHIA COLI /100ML - MF        │ 255609 │
│ ENTÉROCOQUES /100ML-MS              │ 255593 │
│ BACTÉRIES COLIFORMES /100ML-MS      │ 255539 │
│ TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU       │ 255307 │
│ CHLORE TOTAL                        │ 254918 │
│ BACT. AÉR. REVIVIFIABLES À 36°-44H  │ 254690 │
│          ·                          │      · │
│          ·                          │      · │
│          ·                          │      · │
│ BENZO(J)FLUORANTHÈNE                │      1 │
│ GIARDIA SPP                         │      1 │
│ CHLOROTHALONIL MÉTABOLITE R611965   │      1 │
│ RÉSIDU SEC À 180° 

In [7]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb

# Maintenant configurons l'extension SQL pour Jupyter...

%load_ext sql
%sql duckdb:///../../database/data.duckdb

# ...et faisons la même requête SQL en utilisant l'extension SQL pour Jupyter

%sql SELECT libmajparametre, COUNT(*) as count FROM sise_resultats GROUP BY libmajparametre ORDER BY count DESC;

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

Running query in 'duckdb:///../../database/data.duckdb'

libmajparametre,count
PH,338786
CONDUCTIVITÉ À 25°C,265188
TEMPÉRATURE DE L'EAU,258858
CHLORE LIBRE,257631
ESCHERICHIA COLI /100ML - MF,255609
ENTÉROCOQUES /100ML-MS,255593
BACTÉRIES COLIFORMES /100ML-MS,255539
TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,255307
CHLORE TOTAL,254918
BACT. AÉR. REVIVIFIABLES À 36°-44H,254690


In [8]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb

# Enfin, terminons par lister les prélèvements effectués dans une commune donnée

nomcommune = "TOULOUSE"

con.sql(f"""
    SELECT *
    FROM sise_prelevements
    WHERE nomcommuneprinc = '{nomcommune}'
""").show()

┌─────────┬───────────┬───────────────────┬─────────────────┬───────────────┬───────────────────────────┬───────────────┬───────────────┬────────────┬───────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┬──────────┬────────────────────┬───────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┐
│ cddept  │ cdreseau  │ inseecommuneprinc │ nomcommuneprinc │ cdreseauamont │      nomreseauamont       │ pourcentdebit │ referenceprel │  dateprel  │ heureprel │                                             conclusionprel                                              │       ugelib       │ distrlib │       moalib       │ plvconformitebacterio │ plvconformitechimique │ plvconformitereferencebact │ plvconformitereferencechim │
│ varchar │  varchar  │      varchar      │     varchar     │    varchar    │          varchar          │    varchar    │    varchar    │    dat